In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import pandas as pd
import random
from tqdm import tqdm
import concurrent.futures

In [2]:
file_name = 'clean_ascii'
augmentation_multiplier = 5  # 5배 증강

In [3]:
df: pd.DataFrame = pd.read_csv(f'data/{file_name}_remap.csv')
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df.head()

,ID,target,text
0,ynat-v1_train_00000,4,"KT, 2단계 이용 기간의 Q분기 업데이트: 새로운 기능과 개선 사항 공개"
1,ynat-v1_train_00013,4,"애플, 맥북 프로 '배터리 수명 문제' 해결을 위한 소프트웨어 업데이트 출시"
2,ynat-v1_train_00027,4,"인공지능 기반 보안 기술, 기업의 데이터 보호에 혁명을 일으킨다"
3,ynat-v1_train_00037,4,"인공조명 혁신: dcST C, 생산성 향상 위한 고효율 LED 기술 개발"
4,ynat-v1_train_00052,4,한국 시장에서의 전자제품 판매 실적: 얼마나 벌고 있을까?


In [4]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            """
            You are a helpful assistant that rewrites the given korean sentence by replacing some korean words with their synonyms. 
            Please provide only the modified korean sentence as the output.
            """,
        ),
        (
            "human",
            "{input}",
        )
    ]
)

# 동의어 대체 증강

In [9]:
def augment_sentence(original_id, original_target, sentence, n=augmentation_multiplier):
    augmented_data = []
    for _ in range(n):
        llm = ChatOllama(
            model="gemma2:27b",
            seed=random.randint(0, 10000),
            temperature=0.7  # 다양성을 위해 온도를 높게 설정
        )
        chain = prompt | llm
        augmented_sentence = chain.invoke(input={'input': sentence})
        augmented_data.append({
            'ID': original_id,
            'target': original_target,
            'text': augmented_sentence.content.strip()
        })
    return augmented_data

In [10]:
augmented_data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for idx, row in df.iterrows():
        original_id = row['ID']
        original_target = row['target']
        original_text = row['text']
        futures.append(executor.submit(augment_sentence, original_id, original_target, original_text))
    
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        try:
            augmented_data.extend(future.result())
        except Exception as e:
            print(f"Error processing row: {e}")

100%|██████████| 1539/1539 [3:47:05<00:00,  8.85s/it]  


In [11]:
augmented_df = pd.DataFrame(augmented_data)
final_df = pd.concat([df, augmented_df], ignore_index=True)
final_df

,ID,target,text
0,ynat-v1_train_00000,4,"KT, 2단계 이용 기간의 Q분기 업데이트: 새로운 기능과 개선 사항 공개"
1,ynat-v1_train_00013,4,"애플, 맥북 프로 '배터리 수명 문제' 해결을 위한 소프트웨어 업데이트 출시"
2,ynat-v1_train_00027,4,"인공지능 기반 보안 기술, 기업의 데이터 보호에 혁명을 일으킨다"
3,ynat-v1_train_00037,4,"인공조명 혁신: dcST C, 생산성 향상 위한 고효율 LED 기술 개발"
4,ynat-v1_train_00052,4,한국 시장에서의 전자제품 판매 실적: 얼마나 벌고 있을까?
...,...,...,...
9229,ynat-v1_train_02788,0,남원 지역 예술가 모집: 봄>X학%단+분야별 선도력 프로그램 참여자 신청
9230,ynat-v1_train_02788,0,남원 지역 예술가 모집: 봄>X학%단+분야별 선도력 프로그램 참여자 신청
9231,ynat-v1_train_02788,0,남원 지역 예술가 모집: 봄>X학%단+분야별 선도력 프로그램 참여자 신청
9232,ynat-v1_train_02788,0,남원 지역 예술가 모집: 봄>X학%단+분야별 선도력 프로그램 참여자 신청


In [16]:
final_df['text'] = final_df['text'].str.strip()
final_df_unique = final_df.drop_duplicates(subset=['text'], keep='first', ignore_index=True)
final_df_unique.head()


,ID,target,text
0,ynat-v1_train_00000,4,"KT, 2단계 이용 기간의 Q분기 업데이트: 새로운 기능과 개선 사항 공개"
1,ynat-v1_train_00013,4,"애플, 맥북 프로 '배터리 수명 문제' 해결을 위한 소프트웨어 업데이트 출시"
2,ynat-v1_train_00027,4,"인공지능 기반 보안 기술, 기업의 데이터 보호에 혁명을 일으킨다"
3,ynat-v1_train_00037,4,"인공조명 혁신: dcST C, 생산성 향상 위한 고효율 LED 기술 개발"
4,ynat-v1_train_00052,4,한국 시장에서의 전자제품 판매 실적: 얼마나 벌고 있을까?


In [13]:
final_df.to_csv(f'data/{file_name}_{augmentation_multiplier}_times_augmentation.csv')